In [1]:
# Import necessary libraries

import pandas as pd

In [2]:
# Load the dataset and display the first few rows

states_hic = pd.read_csv('data/states_hic.csv')
states_hic.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2016),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
0,Alabama,14.6%,10.1%,-4.5%,215000,2545000,165534,152206,$310,False,799176.0,910775,111599.0,989855
1,Alaska,19.9%,14.9%,-5%,36000,390000,17995,16205,$750,True,122334.0,166625,44291.0,88966
2,Arizona,16.9%,10.8%,-6.1%,410000,3288000,179445,124346,$230,True,1201770.0,1716198,514428.0,1175624
3,Arkansas,17.5%,9.5%,-8%,234000,1365000,63357,56843,$306,True,556851.0,920194,363343.0,606146
4,California,18.5%,8.6%,-9.9%,3826000,19552000,1415428,1239893,$309,True,7755381.0,11843081,4087700.0,5829777


In [3]:
# Display dataset information

states_hic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 14 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   State                                         52 non-null     object 
 1   Uninsured Rate (2010)                         52 non-null     object 
 2   Uninsured Rate (2015)                         52 non-null     object 
 3   Uninsured Rate Change (2010-2015)             52 non-null     object 
 4   Health Insurance Coverage Change (2010-2015)  52 non-null     int64  
 5   Employer Health Insurance Coverage (2015)     52 non-null     int64  
 6   Marketplace Health Insurance Coverage (2016)  52 non-null     int64  
 7   Marketplace Tax Credits (2016)                52 non-null     int64  
 8   Average Monthly Tax Credit (2016)             52 non-null     object 
 9   State Medicaid Expansion (2016)               51 non-null     objec

##### Data Cleaning

###### From the data preview, one thing stands out: the percent and currency symbols in the numeric columns. This formatting makes it impossible to aggregate and perform analysis correctly. Therefore, the first step is to remove the percent and currency symbols from the relevant columns and convert them to proper numeric values. 

In [4]:
# Remove the '%' symbols from selected columns and convert them to numeric types
cols = ["Uninsured Rate (2010)", "Uninsured Rate (2015)", "Uninsured Rate Change (2010-2015)"]

for col in cols:
    states_hic[col] = (
        states_hic[col]
        .astype('string')  # Ensure the column is of string type
        .str.strip()        # Remove leading/trailing whitespace
        .str.rstrip('%')    # Remove trailing '%' symbol
        .astype('float')    # Convert to float type
    )

    # Remove the '$' symbols from Tax credit column and convert it to numeric types

states_hic['Average Monthly Tax Credit (2016)'] = (
        states_hic['Average Monthly Tax Credit (2016)']
        .astype('string')
        .str.strip()
        .str.lstrip('$')
        .astype('float')
)

###### Now it is time to handle missing values. From the data information, three columns contain missing entries. We will first identify the affected rows and then decide whether to look up the values from external sources and impute them, or delete those rows if reliable values cannot be obtained.

In [5]:
# Show rows where 'State Medicaid Expansion (2016)' is missing
missing_expansion = states_hic[states_hic['State Medicaid Expansion (2016)'].isna()]
missing_expansion

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2016),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
51,United States,15.5,9.4,6.1,19304000,172292000,11081330,9389609,291.0,NaN,56392477.0,73532931,16106157.0,57149984


###### This row represents aggregate data for the entire country, so it is reasonable to leave this value as NaN and exclude it from state-level analyses.

In [6]:
missing_enrollment = states_hic[states_hic['Medicaid Enrollment (2013)'].isna()]
missing_enrollment

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2016),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
6,Connecticut,9.1,6.0,-3.1,110000,2148000,102917,80759,357.0,True,NaN,761137,NaN,644136
19,Maine,10.1,8.4,-1.7,22000,702000,75240,63896,342.0,False,NaN,273160,NaN,315160


###### From an external source, Medicaid enrollment in late 2013 was approximately 611,000 for Connecticut and 281,000 for Maine. Let us impute these values in the dataset to replace the missing entries.

In [7]:
# Impute missing values based on external data sources

states_hic['State'] = states_hic['State'].str.strip() # Ensure no leading/trailing whitespace in state names

states_hic.loc[states_hic["State"] == "Connecticut", "Medicaid Enrollment (2013)"] = 611000

states_hic.loc[states_hic["State"] == "Maine", "Medicaid Enrollment (2013)"] = 281000

###### Once Medicaid Enrollment (2013) values for Connecticut and Maine are filled, we can recompute the change instead of imputing it arbitrarily. 

In [8]:
states_hic["Medicaid Enrollment Change (2013-2016)"] = (
    states_hic["Medicaid Enrollment (2016)"] - states_hic["Medicaid Enrollment (2013)"]
)

In [9]:
states_hic.to_csv('data/states_hic_cleaned.csv', index=False)